In [2]:
import "dotenv/config";
import { ChatBedrockConverse } from "@langchain/aws";

const llm = new ChatBedrockConverse({
  model: "us.amazon.nova-micro-v1:0",
  region: Deno.env.BEDROCK_AWS_REGION ?? "us-east-1",
  credentials: {
    secretAccessKey: Deno.env.BEDROCK_AWS_SECRET_ACCESS_KEY ?? "",
    accessKeyId: Deno.env.BEDROCK_AWS_ACCESS_KEY_ID ?? "",
  },
  temperature: 0,
});

[Module: null prototype] { default: {}, "module.exports": {} }

## Building Blocks: The Augmented LLM

In [3]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const searchQuerySchema = z.object({
  searchQuery: z.string().describe("Query that is optimized web search."),
  justification: z.string("Why this query is relevant to the user's request."),
});

// Augment the LLM with schema for structured output
const structuredLlm = llm.withStructuredOutput(searchQuerySchema, {
  name: "searchQuery",
});

// Invoke the augmented LLM
const output = await structuredLlm.invoke(
  "How does Calcium CT score relate to high cholesterol?"
);

console.log(output);

const multiply = tool(
  async ({ a, b }) => {
    return a * b;
  },
  {
    name: "multiply",
    schema: z.object({
      a: z.number("the first number"),
      b: z.number("the second number"),
    }),
  }
);

// Augment the LLM with tools
const llmWithTools = llm.bindTools([multiply]);

// Invoke the LLM with input that triggers the tool call
const message = await llmWithTools.invoke("What is 2 times 3?");

console.log(message.tool_calls);


{
  searchQuery: "relationship between Calcium CT score and high cholesterol",
  justification: "To find information on how Calcium CT score relates to high cholesterol"
}
[
  {
    id: "tooluse_gMCLvG3MSBGq83_j4Qn-vg",
    name: "multiply",
    args: { a: 2, b: 3 },
    type: "tool_call"
  }
]


## Prompt chaining

In [4]:
import { task, entrypoint } from "@langchain/langgraph";

// Tasks

// First LLM call to generate initial joke
const generateJoke = task("generateJoke", async (topic: string) => {
  const msg = await llm.invoke(`Write a short joke about ${topic}`);
  return msg.content;
});

// Gate function to check if the joke has a punchline
function checkPunchline(joke: string) {
  // Simple check - does the joke contain "?" or "!"
  if (joke.includes("?") || joke.includes("!")) {
    return "Pass";
  }
  return "Fail";
}

// Second LLM call to improve the joke
const improveJoke = task("improveJoke", async (joke: string) => {
  const msg = await llm.invoke(
    `Make this joke funnier by adding wordplay: ${joke}`
  );
  return msg.content;
});

// Third LLM call for final polish
const polishJoke = task("polishJoke", async (joke: string) => {
  const msg = await llm.invoke(`Add a surprising twist to this joke: ${joke}`);
  return msg.content;
});

const workflow = entrypoint("jokeMaker", async (topic: string) => {
  const originalJoke = await generateJoke(topic);
  if (checkPunchline(originalJoke) === "Pass") {
    return originalJoke;
  }
  const improvedJoke = await improveJoke(originalJoke);
  const polishJoke = await polishJoke(improvedJoke);
  return polishJoke;
});

const stream = await workflow.stream("cats", { streamMode: "updates" });

for await (const step of stream) {
  console.log(step);
}


{
  generateJoke: "Why did the cat go to the party alone?\n" +
    "\n" +
    "Because it wanted to be the purr-ty one!"
}
{
  jokeMaker: "Why did the cat go to the party alone?\n" +
    "\n" +
    "Because it wanted to be the purr-ty one!"
}
